In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the dataset
train_data = pd.read_csv('./archive/train.csv')
train_data.head()

In [ ]:
# Calculate the proportion of each category
category_proportions = train_data.iloc[:, 3:].mean()

# Print the proportions to check the values
print(category_proportions)
print(train_data.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(x=category_proportions.index, y=category_proportions.values)
plt.title('Proportion of Samples in Each Category')
plt.ylabel('Proportion')
plt.xlabel('Categories')
plt.xticks(rotation=45)
plt.show()

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters (you can tune these)
vocab_size = 10000
max_length = 120
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

# Combining TITLE and ABSTRACT for the input data
all_text = train_data['TITLE'] + ' ' + train_data['ABSTRACT']

# Tokenization
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(all_text)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(all_text)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Prepare target variables
labels = train_data.iloc[:, 3:].values
print(labels)

NameError: name 'train_data' is not defined

In [ ]:
# if on Apple Silicon M1
from tensorflow.keras.optimizers.legacy import Adam

In [ ]:
# otherwise
#from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense

# Model parameters
embedding_dim = 16

# Build the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(labels.shape[1], activation='sigmoid')  # Output layer with one neuron per category
])

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [ ]:
# Training parameters
epochs = 20
batch_size = 64

# Train the model
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                    validation_data=(X_val, y_val), verbose=1)

In [ ]:
# Plotting training and validation accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# save the model
model.save('research_paper_classification3.keras')

In [ ]:
# evaluate the model in % of accuracy
loss, accuracy = model.evaluate(X_val, y_val, verbose=2)

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from sklearn.metrics import classification_report

# Get binary predictions for the validation set
val_predictions = (model.predict(X_val) > 0.5).astype(int)

# Print classification report
print(classification_report(y_val, val_predictions, target_names=category_names))

# Making Predictions on Test Data

In [1]:
# load the model
from tensorflow.keras.models import load_model

model = load_model('research_paper_classification2.keras', compile=False) # model with the best accuracy 77.33%
model.compile()

2023-12-05 15:19:31.907927: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-12-05 15:19:31.907956: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-12-05 15:19:31.907961: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-12-05 15:19:31.907987: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-12-05 15:19:31.908001: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
import pandas as pd

# Load the test dataset
test_data = pd.read_csv('archive/test.csv')

# Combining TITLE and ABSTRACT for the input data
test_text = test_data['TITLE'] + ' ' + test_data['ABSTRACT']

In [10]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Preprocess the test data
test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

NameError: name 'tokenizer' is not defined

In [11]:
# Make predictions
predictions = model.predict(test_padded)

NameError: name 'test_padded' is not defined

In [ ]:
# Applying a threshold to convert probabilities to binary values
threshold = 0.5
number_of_papers = 10
binary_predictions = (predictions > threshold).astype(int)

In [ ]:
# Displaying the first few binary predictions
print(binary_predictions[:number_of_papers])

In [ ]:
# Assuming you have a list of category names corresponding to the order in your label columns
category_names = ['Computer Science', 'Physics', 'Mathematics', 'Statistics', 'Quantitative Biology', 'Quantitative Finance']

# Function to map binary predictions to category names
def map_predictions(binary_preds, category_names):
    pred_labels = []
    for pred in binary_preds:
        labels = [category_names[i] for i, val in enumerate(pred) if val == 1]
        pred_labels.append(labels)
    return pred_labels

# Mapping the first few predictions
mapped_predictions = map_predictions(binary_predictions[:number_of_papers], category_names)

# Display mapped predictions
for i, pred in enumerate(mapped_predictions):
    print(f"Test Sample {i+1}: {pred if pred else 'No Categories Predicted'}")

In [ ]:
# Display predictions with test data for the first few samples
for i in range(number_of_papers):
    print(f"Test Sample {i+1}:")
    print(f"Title: {test_data['TITLE'].iloc[i]}")
    print(f"Abstract: {test_data['ABSTRACT'].iloc[i]}")
    print(f"Predicted Categories: {mapped_predictions[i] if mapped_predictions[i] else 'No Categories Predicted'}")
    print("\n")

In [ ]:
# Save predictions in a dataframe
submission_df = pd.DataFrame(columns=['ID'] + category_names)
submission_df['ID'] = test_data['ID'].values
# add title and abstract columns
submission_df['TITLE'] = test_data['TITLE'].values
submission_df['ABSTRACT'] = test_data['ABSTRACT'].values
# add predictions
submission_df[category_names] = binary_predictions

In [ ]:
# save the dataframe as excel file
submission_df.to_excel('submission.xlsx', index=False)
# display the dataframe with predictions
submission_df.head(20)